# Getting Started with AutoProf

In this notebook you will walk through the very basics of AutoProf functionality. Here you will learn how to make models; how to set them up for fitting; and how to view the results. These core elements will come up every time you use AutoProf, though in future notebooks you will learn how to take advantage of the advanced features in AutoProf.

In [ ]:
import os
import autoprof as ap
import numpy as np
import torch
from astropy.io import fits
import matplotlib.pyplot as plt
%matplotlib inline

## Your first model

In [ ]:
model1 = ap.models.AutoProf_Model(
    name = "model1", # every model must have a unique name
    model_type = "sersic galaxy model", # this specifies the kind of model
    parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "n": 2, "Re": 10, "Ie": 1}, # here we set initial values for each parameter
)

# We can print the model's basic info
print(model1)

In [ ]:
# AutoProf has built in methods to plot relevant information

fig, ax = plt.subplots(figsize = (8,8))
ap.plots.model_image(fig, ax, model1)
plt.show()

## Giving the model a Target

Typically, the main goal when constructing an AutoProf model is to fit to an image. We need to give the model access to the image and some information about it to get started.

In [ ]:
# first let's download an image to play with
hdu = fits.open("https://www.legacysurvey.org/viewer/fits-cutout?ra=36.3684&dec=-25.6389&size=700&layer=ls-dr9&pixscale=0.262&bands=r")
target_data = np.array(hdu[0].data, dtype = np.float64)

# Create a target object with specified pixelscale and zeropoint
target = ap.image.Target_Image(
    data = target_data,
    pixelscale = 0.262,
    zeropoint = 22.5,
)

# The default AutoProf target plotting method uses log scaling in bright areas and histogram scaling in faint areas
fig3, ax3 = plt.subplots(figsize = (8,8))
ap.plots.target_image(fig3, ax3, target)
plt.show()

In [ ]:
# This model now has a target that it will attempt to match
model2 = ap.models.AutoProf_Model(
    name = "model with target", 
    model_type = "sersic galaxy model",
    target = target,
)

# Instead of giving initial values for all the parameters, it is possible to simply call "initialize" and AutoProf 
# will try to guess initial values for every parameter assuming the galaxy is roughly centered. It is also possible
# to set just a few parameters and let AutoProf try to figure out the rest. For example you could give it an initial
# Guess for the center and it will work from there.
model2.initialize()

# Plotting the initial parameters and residuals, we see it gets the rough shape of the galaxy right, but still has some fitting to do
fig4, ax4 = plt.subplots(1, 2, figsize = (16,7))
ap.plots.model_image(fig4, ax4[0], model2)
ap.plots.residual_image(fig4, ax4[1], model2)
plt.show()

In [ ]:
# Now that the model has been set up with a target and initialized with parameter values, it is time to fit the image
model2.fit()

In [ ]:
fig5, ax5 = plt.subplots(1, 2, figsize = (16,7))
ap.plots.model_image(fig5, ax5[0], model2)
ap.plots.residual_image(fig5, ax5[1], model2)
plt.show()

## Giving the model a specific target window

Sometimes an object isn't nicely centered in the image, and may not even be the dominant object in the image. It is therefore nice to be able to specify what part of the image we should analyze.

In [ ]:
model3 = ap.models.AutoProf_Model(
    name = "model with target", 
    model_type = "sersic galaxy model",
    target = target,
    window = [[480, 590],[555, 665]],
)

fig6, ax6 = plt.subplots(figsize = (8,8))
ap.plots.model_window(fig6, ax6, model3)
plt.show()

In [ ]:
model3.initialize()
model3.fit()

In [ ]:
# Note that when only a window is fit, the default plotting methods will only show that window
fig7, ax7 = plt.subplots(1, 2, figsize = (16,7))
ap.plots.model_image(fig7, ax7[0], model3)
ap.plots.residual_image(fig7, ax7[1], model3)
plt.show()

## Setting parameter constraints

A common feature of fitting parameters is that they have some constraint on their behaviour and cannot be sampled at any value from (-inf, inf). AutoProf circumvents this by remapping any constrained parameter to a space where it can take any real value, at least for the sake of fitting. For most parameters these constraints are applied by default; for example the axis ratio q is required to be in the range (0,1). Other parameters, such as the position angle (PA) are cyclic, they can be in the range (0,pi) but also can wrap around. It is possible to manually set these constraints while constructing a model.

In general adding constraints makes fitting more difficult. There is a chance that the fitting process runs up against a constraint boundary and gets stuck. However, sometimes adding constraints is necessary and so the capability is included.

In [ ]:
# here we make a sersic model that can only have q and n in a narrow range
# Also, we give PA and initial value and lock that so it does not change during fitting
constrained_param_model = ap.models.AutoProf_Model(
    name = "constrained parameters", model_type = "sersic galaxy model", 
    parameters = {
        "q": {"limits": [0.4,0.6]}, 
        "n": {"limits": [2,3]}, 
        "PA": {"value": 60*np.pi/180, "locked": True},
    }
)

In [ ]:
# general constraints can be added as well, this is in the form of a function which is zero when satisfied and 
# positive when violated. AutoProf will account for the constraint, though may violate it by a small amount.
constrained_model = ap.models.AutoProf_Model(
    name = "general constraints", model_type = "sersic galaxy model",
    constraints = [
        lambda M: torch.max(0, M["n"] - M["q"]), # n > q
        lambda M: torch.abs(M["center"].value[0] - M["center"].value[1]**2), # x = y^2
    ],
)

The constraints are provided as a list of functions. The functions take a model as input and return a real number which goes from 0,inf where 0 means the constraint is satisfied. In the above case there is a constraint that the sersic index is larger than the axis ratio; there is another constraint that the x-coordinate of the center is equal to the y-coordinate squared. These are absurd constraints, but in compound models it can be useful to set say the radius of one model to be larger than the another.

Note as well that the constraint functions have to use pytorch since they will be included in gradient descent and/or GPU calculations.

## PSF convolution

An important part of astronomical image analysis is accounting for PSF effects. To that end, AutoProf includes a number of approaches to handle PSF convolution. The main concept is that AutoProf will convolve its model with a PSF before comparing against an image. The PSF behaviour of a model is determined by the *psf_mode* parameter which can be set before fitting.

In [ ]:
# first a psf is needed, this is stored with the target object
xx, yy = np.meshgrid(np.linspace(-5,5,11), np.linspace(-5,5,11))
PSF = np.exp(-(xx**2 + yy**2)/2**2)
PSF /= np.sum(PSF)
target = ap.image.Target_Image(
    data = target_data,
    pixelscale = 0.262,
    zeropoint = 22.5,
    psf = PSF,
)

model_psf = ap.models.AutoProf_Model(
    name = "model with psf", 
    model_type = "sersic galaxy model",
    target = target,
    psf_mode = "window", # the model will convolve the PSF only in a window around the center of the model
    psf_window_size = 100, # The window around the center of the model will be 100 pixels across
)
print("upon creation: ", model_psf.psf_mode)

# We can also set the psf mode separately
model_psf.psf_mode = "full" # now the full window will be PSF convolved
print("modified for this model: ", model_psf.psf_mode)

## Basic things to do with a model

Now that we know how to create a model and fit it to an image, lets get to know the model a bit better.

In [ ]:
# Save the model to a file
model2.save() # will default to save as AutoProf.yaml
with open("AutoProf.yaml", "r") as f:
    print(f.read()) # show what the saved file looks like

In [ ]:
# load a model from a file

# note that the target still must be specified, only the parameters are saved
model4 = ap.models.AutoProf_Model(name = "no name", filename = "AutoProf.yaml", target = target)
print(model4) # can see that it has been constructed with all the same parameters as the saved model2.

In [ ]:
# Plot the surface brightness profile

fig8, ax8 = plt.subplots(figsize = (8,8))
ap.plots.galaxy_light_profile(fig8, ax8, model4)
plt.legend()
plt.show()

In [ ]:
# Give the model new parameter values manually

print("parameter input order: ", model4.parameter_order) # use this to see what order you have to give the parameters as input

# plot the old model
fig9, ax9 = plt.subplots(1,2,figsize = (16,7))
ap.plots.model_image(fig9, ax9[0], model4)
T = ax9[0].set_title("parameters as loaded")

# update and plot the new parameters
new_parameters = torch.tensor([75, 110, 0.4, 20*np.pi/180, 3, 25, 0.12]) # note that the center parameter needs two values as input
model4.startup() # startup must be called before optimization, or any other activity in which parameters are updated
model4.full_sample(new_parameters) # full_sample will update the parameters, then run sample and return the model image 
ap.plots.model_image(fig9, ax9[1], model4)
T = ax9[1].set_title("new parameter values")

In [ ]:
# Access the model image pixels directly

fig2, ax2 = plt.subplots(figsize = (8,8))
model4.sample() # Tell the model to sample an image using the current parameter values/settings

pixels = model4.model_image.data.detach().cpu().numpy()# model1.model_image.data is the pytorch stored model image pixel values. Calling detach().cpu().numpy() is needed to get the data out of pytorch and in a usable form

im = plt.imshow(
    np.log10(pixels), # take log10 for better dynamic range
    origin = "lower",
    cmap = ap.plots.visuals.cmap_grad, # gradient colourmap default for AutoProf
)
plt.colorbar(im)
plt.show()

In [ ]:
# Models can be constructed by providing model_type, or by creating the desired class directly

#                     notice this is no longer "AutoProf_Model"
model1_v2 = ap.models.Sersic_Galaxy(
    name = "model1 v2",
    parameters = {"center": [50,50], "q": 0.6, "PA": 60*np.pi/180, "n": 2, "Re": 10, "Ie": 1},
)

# This will be the same as model1
print(model1_v2)

## Using GPU acceleration

This one is easy! If you have a cuda enabled GPU available, AutoProf will just automatically detect it and use that device. 

In [ ]:
# check if AutoProf has detected your GPU
print(ap.models.AutoProf_Model.device) # most likely this will say "cpu" unless you already have a cuda GPU, in which case it should say "cuda:0"

In [ ]:
# If you have a GPU but want to use the cpu for some reason, just set:
ap.models.AutoProf_Model.device = "cpu"
# before creating your models